In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from torchvision.io import read_video
from torchvision.transforms.functional import resize,normalize
import ultralytics
import depth_pro

ultralytics.checks()
model = ultralytics.YOLO("yolo11x-pose.pt")

Ultralytics 8.3.85 🚀 Python-3.13.2 torch-2.6.0+cu126 CUDA:0 (NVIDIA RTX A2000 8GB Laptop GPU, 8192MiB)
Setup complete ✅ (24 CPUs, 15.5 GB RAM, 126.7/1006.9 GB disk)


In [2]:
# Load depth model and preprocessing transform
depth_model, transform = depth_pro.create_model_and_transforms(device="cuda",precision=torch.half)
depth_model.eval()
print("all good")

all good


In [ ]:
# Load video to tensor and preprocess to the specs depth pro uses as input
v_frames = read_video("Homebrew-video/Low-quality/IMG_5347.MP4",output_format="TCHW")[0].to("cuda").half()/255
v_frames = normalize(v_frames,[0.5,0.5,0.5],[0.5,0.5,0.5],inplace=True)

/root/miniconda3/envs/SUSTxSUBSEQUENT/lib/python3.13/site-packages/torchvision/io/video.py:197: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


In [ ]:
# Run depth inference on a single frame
prediction = depth_model.infer(v_frames[:9])
depth = prediction["depth"]  # Depth in [m].
focallength_px = prediction["focallength_px"]  # Focal length in pixels.

In [ ]:
print(depth)
print(depth.shape)
print(focallength_px)
print(focallength_px.shape)

In [4]:
# Load YOLO object detections
results = model.track(source="Homebrew-video/Low-quality/IMG_5347.MP4",classes=[0])



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/368) /root/SUSTAI_X_SUBSEQUENTAI_Group_project/Homebrew-video/Low-quality/IMG_5347.MP4: 384x640 4 persons, 65.3ms
video 1/1 (frame 2/368) /root/SUSTAI_X_SUBSEQUENTAI_Group_project/Homebrew-video/Low-quality/IMG_5347.MP4: 384x640 4 persons, 62.6ms
video 1/1 (frame 3/368) /root/SUSTAI_X_SUBSEQUENTAI_Group_project/Homebrew-video/Low-quality/IMG_5347.MP4: 384x640 4 persons, 71.9ms
video 1/1 (frame 4/368) /root/SUSTAI_X_SUBSEQUENTAI_Group

In [ ]:
frame_no=69
aspect_ratio = 1280/720

for i,result in enumerate(results):
    if i!=frame_no:continue

    # Load and preprocess an image.
    d_img = transform(result.orig_img)
    f_px = depth_pro.utils.fpx_from_f35(d_img.shape[2],d_img.shape[1],26) # Converts focal length from my phone camera to fpx.
    
    # Run inference.
    prediction = depth_model.infer(d_img,f_px)
    depth = prediction["depth"]  # Depth in [m].
    focallength_px = prediction["focallength_px"]  # Focal length in pixels.

    for j,box in enumerate(result.boxes):
        track_id = box.id ;print(f"person number {track_id.item()}")
        xy = box.xyxy[0]#; print(xy); print(xy[:2],xy[2:])
        c = torch.tensor([ (xy[:2][0]+xy[2:][0])/2, (xy[2:][-1]+xy[:2][-1])/2 ], device="cuda") #;print("centroids",c)
        w,h = box.xywh[0][2],box.xywh[0][3] ;print("image width & height (px)",w.item(),h.item())
        f = focallength_px; print("focal length (px)", f.item())
        d = depth[int(c[1].item())][int(c[0].item())] ;print("depth value at centroid (m)",d.item())
        pw = w*d/f ;print("object width (m)",pw.item())
        ph = h*d/f ;print("object height (m)",ph.item())

    #print(i.keypoints[0])
    

person number 1.0
image width & height (px) 60.53936767578125 165.62716674804688
focal length (px) 882.5216396465553
depth value at centroid (m) 7.80078125
object width (m) 0.5351192951202393
object height (m) 1.4640108346939087
person number 2.0
image width & height (px) 81.12017822265625 149.51852416992188
focal length (px) 882.5216396465553
depth value at centroid (m) 8.2109375
object width (m) 0.7547380924224854
object height (m) 1.3911129236221313
person number 3.0
image width & height (px) 49.8160400390625 169.93002319335938
focal length (px) 882.5216396465553
depth value at centroid (m) 7.9765625
object width (m) 0.4502560794353485
object height (m) 1.5358914136886597
person number 4.0
image width & height (px) 65.7518310546875 150.223876953125
focal length (px) 882.5216396465553
depth value at centroid (m) 9.4765625
object width (m) 0.706046462059021
object height (m) 1.6131116151809692
